### 가게별 UserID, ItemID, Rating, Timestamp 뽑기
- user_review_id = {}
- review_json = {}
- image_json = {}
- item_meta = {}

In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException

In [2]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])

In [3]:
user_review_id = {}
review_json = {}
image_json = {}
item_meta = {}

In [5]:
df = pd.read_csv("C:/Users/정예림/Desktop/추천시스템 프로젝트/data/구별/nowon.csv", encoding="UTF-8-sig")
df=df[["상호명", "상권업종중분류명", "상권업종소분류명", "시군구명", "법정동명", "도로명주소", "kakao_keyword", "kakao_url"]]
df

,상호명,상권업종중분류명,상권업종소분류명,시군구명,법정동명,도로명주소,kakao_keyword,kakao_url
0,놀부부대찌개철판구이성북역점,한식,부대찌개/섞어찌개,노원구,월계동,서울특별시 노원구 석계로 101,월계동 놀부부대찌개철판구이성북역점,https://place.map.kakao.com/19197518
1,오딧세이월계점,유흥주점,호프/맥주,노원구,월계동,서울특별시 노원구 월계로 366,월계동 오딧세이월계점,NaN
2,서울과학기술대학교제2학생식당,기타음식업,구내식당/자급식음식점,노원구,공릉동,서울특별시 노원구 공릉로 232,공릉동 서울과학기술대학교제2학생식당,NaN
3,동대문닭한마리칼국수,한식,한식/백반/한정식,노원구,공릉동,서울특별시 노원구 공릉로38길 8-9,공릉동 동대문닭한마리칼국수,https://place.map.kakao.com/15823847
4,커피안바우,커피점/카페,커피전문점/카페/다방,노원구,월계동,서울특별시 노원구 덕릉로60길 252,월계동 커피안바우,NaN
...,...,...,...,...,...,...,...,...
4246,범피버거,패스트푸드,패스트푸드,노원구,공릉동,서울특별시 노원구 공릉로41길 12,공릉동 범피버거,https://place.map.kakao.com/2038251139
4247,휴랩,분식,라면김밥분식,노원구,공릉동,서울특별시 노원구 동일로173가길 140,공릉동 휴랩,NaN
4248,식껍,한식,갈비/삼겹살,노원구,중계동,서울특별시 노원구 한글비석로20길 38,중계동 식껍,https://place.map.kakao.com/1763761834
4249,기찻길풍경,한식,곱창/양구이전문,노원구,공릉동,서울특별시 노원구 공릉로39길 21,공릉동 기찻길풍경,https://place.map.kakao.com/1137642618


In [6]:
# kakao_url 이 없는 것 제외
nowon = df.dropna(axis=0)
nowon

,상호명,상권업종중분류명,상권업종소분류명,시군구명,법정동명,도로명주소,kakao_keyword,kakao_url
0,놀부부대찌개철판구이성북역점,한식,부대찌개/섞어찌개,노원구,월계동,서울특별시 노원구 석계로 101,월계동 놀부부대찌개철판구이성북역점,https://place.map.kakao.com/19197518
3,동대문닭한마리칼국수,한식,한식/백반/한정식,노원구,공릉동,서울특별시 노원구 공릉로38길 8-9,공릉동 동대문닭한마리칼국수,https://place.map.kakao.com/15823847
6,배스킨라빈스공릉점,패스트푸드,아이스크림판매,노원구,공릉동,서울특별시 노원구 공릉로 198,공릉동 배스킨라빈스공릉점,https://place.map.kakao.com/19153143
7,강고집즉석바지락칼국수,분식,국수/만두/칼국수,노원구,공릉동,서울특별시 노원구 공릉로38길 8-9,공릉동 강고집즉석바지락칼국수,https://place.map.kakao.com/20741840
9,아자카야,한식,한식/백반/한정식,노원구,중계동,서울특별시 노원구 덕릉로83길 20-10,중계동 아자카야,https://place.map.kakao.com/1483918776
...,...,...,...,...,...,...,...,...
4245,꼬꼬방집으로삼겹,한식,갈비/삼겹살,노원구,상계동,서울특별시 노원구 상계로17길 11,상계동 꼬꼬방집으로삼겹,https://place.map.kakao.com/1357999307
4246,범피버거,패스트푸드,패스트푸드,노원구,공릉동,서울특별시 노원구 공릉로41길 12,공릉동 범피버거,https://place.map.kakao.com/2038251139
4248,식껍,한식,갈비/삼겹살,노원구,중계동,서울특별시 노원구 한글비석로20길 38,중계동 식껍,https://place.map.kakao.com/1763761834
4249,기찻길풍경,한식,곱창/양구이전문,노원구,공릉동,서울특별시 노원구 공릉로39길 21,공릉동 기찻길풍경,https://place.map.kakao.com/1137642618


In [7]:
item = nowon[:5]
item

,상호명,상권업종중분류명,상권업종소분류명,시군구명,법정동명,도로명주소,kakao_keyword,kakao_url
0,놀부부대찌개철판구이성북역점,한식,부대찌개/섞어찌개,노원구,월계동,서울특별시 노원구 석계로 101,월계동 놀부부대찌개철판구이성북역점,https://place.map.kakao.com/19197518
3,동대문닭한마리칼국수,한식,한식/백반/한정식,노원구,공릉동,서울특별시 노원구 공릉로38길 8-9,공릉동 동대문닭한마리칼국수,https://place.map.kakao.com/15823847
6,배스킨라빈스공릉점,패스트푸드,아이스크림판매,노원구,공릉동,서울특별시 노원구 공릉로 198,공릉동 배스킨라빈스공릉점,https://place.map.kakao.com/19153143
7,강고집즉석바지락칼국수,분식,국수/만두/칼국수,노원구,공릉동,서울특별시 노원구 공릉로38길 8-9,공릉동 강고집즉석바지락칼국수,https://place.map.kakao.com/20741840
9,아자카야,한식,한식/백반/한정식,노원구,중계동,서울특별시 노원구 덕릉로83길 20-10,중계동 아자카야,https://place.map.kakao.com/1483918776


In [8]:
for i, row in item.iterrows() :
    url = item["kakao_url"][i]
    print(i, url,item["상호명"][i])

0 https://place.map.kakao.com/19197518 놀부부대찌개철판구이성북역점
3 https://place.map.kakao.com/15823847 동대문닭한마리칼국수
6 https://place.map.kakao.com/19153143 배스킨라빈스공릉점
7 https://place.map.kakao.com/20741840 강고집즉석바지락칼국수
9 https://place.map.kakao.com/1483918776 아자카야


In [43]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path=r'C:\Users\정예림\Desktop\텍스트마이닝\chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 

In [44]:
for i, row in item.iterrows() :
    
    url = item["kakao_url"][i]
    print(i,"번째 크롤링 중 : ",item["상호명"][i] , sep=" ")
    
    # 상세보기 접속
    driver.get(url)
    
    # html  파싱
    html = driver.page_source
    time.sleep(3)
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(3)
    
    # 리뷰 갯수 --> 몇 페이지인지 알기위해 
    try :
        driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[2]').click()
        rating_num = driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_sorting > a > span.color_b').text
        rating_num = int(rating_num)
        print(rating_num)
        
        # 페이지 수 --> 리뷰 수가 5개 이하이면 한 페이지, 6개 이상이면 n개의 페이지
    
        if rating_num > 0 and rating_num <=5 :
            page_num = 1
            print(page_num)
            
            for j in range(1,page_num) :
                # 해당 페이지의 리뷰리스트 가져오기
                # 리뷰 리스트는 list_evaluation 클래스 안에 li 태그에 있음
                review_lists = driver.find_elements_by_css_selector("#mArticle > div.cont_evaluation > div.evaluation_review > ul > li")
        
                #1-5 페이지
                if j <= 4 :
                    #리뷰 리스트가 0이 아니면
                    if len(review_lists) != 0 :
                        for m, review in enumerate(review_lists) : ## --->  데이터 긁어오는거 안됨... 해결해랴!!!
                            data=[]
                            UserID = soup.select('.append_item > a[data-userid]')[0].get('data-userid') #UserID
                            ItemID = item["상호명"][i]                                                  #ItemID
                            Rating = int(soup.select('.grade_star > em')[0].text)                       #Rating
                            Timestamp = soup.select('.time_write')[0].text                              #Timestamp
                            data = UserID, ItemID, Rating, Timestamp      
                            print(data)
                            rating_df.append(data)                                                      #UserID, ItemID, Rating, Timestamp를 rating_df에 append

                        try :
                            # 다음 페이지로 넘어가기
                            driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child('+str(j+2)+')').click()
                            time.sleep(2)
                        except (NoSuchElementException, ElementNotInteractableException) :
                            continue

                elif j == 5:
                    #리뷰 리스트가 0이 아니면
                    if len(review_lists) != 0 :
                        for m, review in enumerate(review_lists) :
                            data=[]
                            UserID = soup.select('.append_item > a[data-userid]')[0].get('data-userid') #UserID
                            ItemID = item["상호명"][i]                                                  #ItemID
                            Rating = int(soup.select('.grade_star > em')[0].text)                       #Rating
                            Timestamp = soup.select('.time_write')[0].text                              #Timestamp
                            data = UserID, ItemID, Rating, Timestamp
                            print(data)
                            rating_df.append(data)                                                      #UserID, ItemID, Rating, Timestamp를 rating_df에 append

                    try :
                        # 다음 페이지로 넘어가기
                        driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a.btn_next') #다음 버튼 누르기
                        time.sleep(2)
                    except (NoSuchElementException, ElementNotInteractableException) :
                        continue
                    
        else :
            page_num = int(math.ceil(rating_num / 5))
            print(page_num)
            
            for j in range(1,page_num) :
                # 해당 페이지의 리뷰리스트 가져오기
                # 리뷰 리스트는 list_evaluation 클래스 안에 li 태그에 있음
                review_lists = driver.find_elements_by_css_selector("#mArticle > div.cont_evaluation > div.evaluation_review > ul > li")

                #1-5 페이지
                if j <= 4 :
                    #리뷰 리스트가 0이 아니면
                    if len(review_lists) != 0 :
                        for m, review in enumerate(review_lists) :
                            data=[]
                            UserID = soup.select('.append_item > a[data-userid]')[0].get('data-userid') #UserID
                            ItemID = item["상호명"][i]                                                  #ItemID
                            Rating = int(soup.select('.grade_star > em')[0].text)                       #Rating
                            Timestamp = soup.select('.time_write')[0].text                              #Timestamp
                            data = UserID, ItemID, Rating, Timestamp      
                            print(data)
                            rating_df.append(data)                                                      #UserID, ItemID, Rating, Timestamp를 rating_df에 append

                        try :
                            # 다음 페이지로 넘어가기
                            driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child('+str(j+2)+')').click()
                            time.sleep(2)
                        except (NoSuchElementException, ElementNotInteractableException) :
                            continue
        
                elif j == 5:
                    #리뷰 리스트가 0이 아니면
                    if len(review_lists) != 0 :
                        for m, review in enumerate(review_lists) :
                            data=[]
                            UserID = soup.select('.append_item > a[data-userid]')[0].get('data-userid') #UserID
                            ItemID = item["상호명"][i]                                                  #ItemID
                            Rating = int(soup.select('.grade_star > em')[0].text)                       #Rating
                            Timestamp = soup.select('.time_write')[0].text                              #Timestamp
                            data = UserID, ItemID, Rating, Timestamp
                            print(data)
                            rating_df.append(data)                                                      #UserID, ItemID, Rating, Timestamp를 rating_df에 append

                    try :
                        # 다음 페이지로 넘어가기
                        driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a.btn_next') #다음 버튼 누르기
                        time.sleep(2)
                    except (NoSuchElementException, ElementNotInteractableException) :
                        continue
    
        
                 #6페이지 이상
                elif j >= 6 :
            
                    #리뷰 리스트가 0이 아니면
                    if len(review_lists) != 0 :
                        for m, review in enumerate(review_lists) :
                            data=[]
                            UserID = soup.select('.append_item > a[data-userid]')[0].get('data-userid')
                            ItemID = item["상호명"][i]
                            Rating = int(soup.select('.grade_star > em')[0].text)
                            Timestamp = soup.select('.time_write')[0].text
                            data = UserID, ItemID, Rating, Timestamp
                            print(data)
                            rating_df.append(data)


                        # 다음 페이지로 넘어가기
                        try :
                            if (j%5) == 1 :
                                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(4)').click()
                                time.sleep(2)
                            elif (j%5) == 2 :
                                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(5)').click()
                                time.sleep(2)
                            elif (j%5) == 3 :
                                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(6)').click()
                                time.sleep(2)
                            elif (j%5) == 4 :
                                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(7)').click()
                                time.sleep(2)
                            else :
                                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a.btn_next').click()
                                time.sleep(2)
                        except (NoSuchElementException, ElementNotInteractableException) :
                            continue
    
            else : 
                print("리뷰가 없습니다.")
        
    except (NoSuchElementException, ElementNotInteractableException) :
        print("리뷰가 없습니다")
        continue 
    
    
    

0 번째 크롤링 중 :  놀부부대찌개철판구이성북역점
3
1
3 번째 크롤링 중 :  동대문닭한마리칼국수
리뷰가 없습니다
6 번째 크롤링 중 :  배스킨라빈스공릉점
6
2


IndexError: list index out of range

In [40]:
rating_df

,UserID,ItemID,Rating,Timestamp
